# **Pipeline RAG com LangChain - embeddings → retriever → LLM**

In [1]:
# Notebook: Pipeline RAG com LangChain - embeddings → retriever → LLM

# Objetivo:
# Demonstrar um pipeline básico de RAG utilizando LangChain, onde:
# 1. Criamos embeddings de um conjunto de documentos textuais.
# 2. Construímos um retriever para buscar informações relevantes.
# 3. Utilizamos um LLM para gerar respostas contextuais a partir do conteúdo recuperado.

# IMPORTANTE:
# - Para rodar este notebook você precisa ter:
#   * Python 3.8+
#   * LangChain instalado (`pip install langchain`)
#   * OpenAI SDK instalado (`pip install openai`)
#   * Ter uma chave de API OpenAI configurada na variável ambiente OPENAI_API_KEY
# - Este é um exemplo simplificado para entendimento do pipeline.

# **1. IMPORTAÇÕES E INSTALAÇÕES**

In [2]:
"""
Instalação de dependências e importação de módulos necessários para o pipeline RAG.

1. Instala pacotes via pip:
    - faiss-cpu: biblioteca para busca vetorial eficiente (FAISS em CPU).
    - sentence-transformers: modelos de embeddings pré-treinados da Hugging Face.
    - langchain: núcleo da biblioteca LangChain.
    - langchain-community: integrações e conectores mantidos pela comunidade LangChain.

2. Importações principais:
    - OpenAIEmbeddings (langchain.embeddings):
        Geração de embeddings usando a API da OpenAI.
    - FAISS (langchain.vectorstores / langchain_community.vectorstores):
        Indexação vetorial com FAISS (há duas implementações, a "core" e a da comunidade).
    - OpenAI (langchain.llms):
        Interface para LLMs da OpenAI (ex.: GPT-3.5, GPT-4).
    - RetrievalQA (langchain.chains):
        Chain que combina retriever + LLM para perguntas e respostas (RAG).
    - HuggingFaceEmbeddings (langchain_community.embeddings):
        Alternativa open source para gerar embeddings localmente.
    - os (módulo padrão do Python):
        Usado para ler variáveis de ambiente (como OPENAI_API_KEY).

Notas:
- O uso de `!pip install` é típico em notebooks (Google Colab, Jupyter).
- Recomenda-se fixar versões em ambientes de produção para evitar que mudanças de API quebrem o código.
"""

!pip install -q faiss-cpu sentence-transformers langchain langchain-community

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
import os

# **2. CRIANDO UM CONJUNTO DE DOCUMENTOS DE EXEMPLO**

In [3]:
"""
Define a lista de documentos de exemplo a serem indexados no pipeline RAG.

Descrição:
- Cada item da lista `docs` é uma string representando um conceito central
  sobre ciência de dados, NLP e RAG.
- Esses documentos simulam uma base de conhecimento reduzida, usada para
  demonstração do fluxo embeddings → FAISS → retriever → LLM.

Conteúdo:
    - Churn: cancelamento ou abandono de clientes.
    - NPS (Net Promoter Score): métrica de lealdade de clientes.
    - LangChain: biblioteca para integrar LLMs a outras fontes de dados.
    - RAG (Retrieval-Augmented Generation): técnica de busca + geração.
    - Embeddings: vetores semânticos que representam significado textual.
    - Pipeline básico de RAG: embeddings → retriever → LLM.

Saída:
    docs (list[str]): base textual que será usada para criar embeddings
    e treinar o índice FAISS no exemplo.

Notas:
- Em um caso real, `docs` poderia vir de arquivos externos (PDFs, CSVs, bases SQL).
- Quanto maior e mais diversificada a base, mais robusto será o retriever.
"""

docs = [
    "O churn é o cancelamento ou abandono de clientes em um serviço ou produto.",
    "NPS, ou Net Promoter Score, mede a lealdade dos clientes através da pergunta: 'Você recomendaria nossa empresa a um amigo?'",
    "LangChain é uma biblioteca para construir aplicações que usam modelos de linguagem large (LLMs) integrados a outras fontes de dados.",
    "RAG, Retrieval-Augmented Generation, conecta modelos de linguagem a bases de conhecimento através de embeddings e mecanismos de busca para melhorar respostas.",
    "Embeddings representam texto em vetores numéricos que capturam significado semântico, possibilitando busca eficiente por similaridade.",
    "O pipeline básico de RAG inclui: criação de embeddings, uso do retriever para buscar documentos relevantes e geração da resposta pelo LLM.",
]

# **3. CONSTRUÇÃO DOS EMBEDDINGS DOS DOCUMENTOS**

In [4]:
"""
Criação dos embeddings e construção do índice vetorial FAISS.

Etapas:
1. Inicializa um modelo de embeddings semânticos local usando Hugging Face.
   - Modelo: "sentence-transformers/all-MiniLM-L6-v2"
   - Esse modelo gera vetores densos de 384 dimensões, balanceando qualidade e velocidade.
   - Alternativa: usar OpenAIEmbeddings, se houver chave API configurada.

2. Constrói um índice FAISS a partir da lista de documentos (`docs`).
   - `FAISS.from_texts(docs, embeddings)` transforma cada documento em vetor
     e armazena em uma estrutura de busca vetorial eficiente.
   - Esse índice permite consultas de similaridade para recuperação posterior.

Args:
    docs (list[str]): lista de documentos em texto plano que serão indexados.

Saídas:
    embeddings (HuggingFaceEmbeddings): modelo de geração de embeddings semânticos.
    vectorstore (FAISS): índice FAISS com embeddings prontos para busca de similaridade.

Notas:
- Para datasets maiores, considere salvar o índice com `vectorstore.save_local(path)`
  e recarregar depois com `FAISS.load_local(path, embeddings)`.
- O modelo MiniLM é leve, ideal para protótipos e uso em CPU; versões maiores podem
  trazer mais qualidade, mas exigem GPU.
"""

print("Criando embeddings dos documentos com OpenAIEmbeddings...")

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Criar vetor store FAISS para indexar e buscar similaridades
vectorstore = FAISS.from_texts(docs, embeddings)
print(f"Documentos indexados: {len(docs)}")

Criando embeddings dos documentos com OpenAIEmbeddings...


/tmp/ipython-input-2911721343.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or 

Documentos indexados: 6


# **4. CRIAR O RETRIEVER A PARTIR DO VECTORSTORE**

In [5]:
"""
Cria o retriever a partir do índice vetorial FAISS.

Um retriever é responsável por recuperar os documentos mais relevantes
a partir de uma consulta, com base na similaridade dos embeddings.

Configuração usada:
    search_type (str): "similarity"
        - Calcula a proximidade vetorial entre a query e os documentos.
        - Outros tipos possíveis em LangChain: "mmr" (Maximal Marginal Relevance).
    search_kwargs (dict): {"k": 2}
        - Número de documentos mais semelhantes a serem retornados (top-k).
        - Ajustar `k` pode equilibrar entre precisão (baixo k) e cobertura/contexto (alto k).

Saída:
    retriever (BaseRetriever): objeto compatível com LangChain que pode ser
    passado para `RetrievalQA` ou outras chains.

Notas:
- Este retriever é a ponte entre embeddings (docs) e o LLM.
- Se `k` for muito baixo, pode perder contexto; se for muito alto, pode adicionar ruído.
"""

retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 2}
)
print("Retriever pronto (k=2).")

Retriever pronto (k=2).


# **5. CONFIGURAR O MODELO LLM**

In [6]:
"""
Configuração do LLM (Language Model) para o pipeline RAG.

Este bloco implementa um fallback local usando Hugging Face caso a chave da OpenAI
não esteja configurada no ambiente. O modelo escolhido é o `google/flan-t5-base`,
um modelo seq2seq leve, adequado para CPU ou GPU no Google Colab.

Passos:
1. Verifica se a variável de ambiente `OPENAI_API_KEY` está definida.
   - Se sim, o notebook poderia configurar um LLM da OpenAI (não implementado aqui).
   - Se não, usa o FLAN-T5 como LLM local.
2. Carrega o tokenizer e o modelo pré-treinado da Hugging Face.
3. Cria um `pipeline` de geração de texto (`text2text-generation`) limitado a 256 tokens.
4. Encapsula o pipeline no wrapper `HuggingFacePipeline` do LangChain, para integração uniforme.

Args principais:
    model_name (str): nome do modelo Hugging Face a ser carregado.
    max_new_tokens (int): limite de tokens gerados por resposta (ajustável conforme GPU/CPU).

Saída:
    llm (HuggingFacePipeline): instância de LLM integrada ao LangChain, pronta para uso no RetrievalQA.

Notas:
- Para respostas mais longas, aumente `max_new_tokens`.
- Em máquinas com GPU, pode-se mover o modelo para CUDA:
      mdl = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")
      gen_pipe = pipeline(..., device=0)
"""

import os
USE_OPENAI = "OPENAI_API_KEY" in os.environ and len(os.environ["OPENAI_API_KEY"].strip()) > 0

print("Configurando o LLM local (Hugging Face - FLAN-T5 base)...")
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_community.llms import HuggingFacePipeline

model_name = "google/flan-t5-base"  # leve; bom p/ CPU/GPU do Colab
tok = AutoTokenizer.from_pretrained(model_name)
mdl = AutoModelForSeq2SeqLM.from_pretrained(model_name)
gen_pipe = pipeline("text2text-generation", model=mdl, tokenizer=tok, max_new_tokens=256)

llm = HuggingFacePipeline(pipeline=gen_pipe)

Configurando o LLM local (Hugging Face - FLAN-T5 base)...


Device set to use cuda:0
/tmp/ipython-input-425815035.py:13: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=gen_pipe)


# **6. CRIAR A CHAIN DE RAG: COMBINA RETRIEVER E LLM PARA RESPOSTAS**

In [7]:
"""
Cria a chain de RAG (Retrieval-Augmented Generation) usando LangChain.

A classe `RetrievalQA` combina:
- Um retriever (responsável por buscar documentos relevantes no índice vetorial).
- Um LLM (responsável por gerar a resposta contextualizada).

Args configurados:
    llm (BaseLanguageModel): Modelo de linguagem já inicializado
        (ex.: OpenAI, HuggingFacePipeline).
    retriever (BaseRetriever): Retriever baseado em FAISS configurado na célula anterior.
    return_source_documents (bool): 
        - False → retorna apenas a resposta gerada.
        - True  → retorna também os documentos de origem usados na resposta.

Saída:
    qa_chain (RetrievalQA): instância pronta para receber queries no formato
    `qa_chain.invoke({"query": "sua pergunta"})`.

Notas:
- Este é o núcleo do pipeline RAG: conecta embeddings → retriever → LLM.
- Se `return_source_documents=True`, pode ser útil para depuração e explicabilidade.
"""

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,             # usa o FAISS retriever criado antes
    return_source_documents=False    # defina True se quiser retornar também os docs relevantes
)
print("RAG chain pronta.")

RAG chain pronta.


# **7. REALIZAR PERGUNTAS E GERAR RESPOSTAS USANDO O PIPELINE RAG**

In [8]:
"""
Executa um ciclo de Perguntas & Respostas usando a chain de RAG.

Esta célula:
- Define perguntas de teste (FAQ rápida).
- Invoca a chain `qa_chain` com o método moderno `invoke` (substitui `run`, que está deprecado).
- Imprime o resultado de cada pergunta com um separador.

Pré-requisitos:
- `qa_chain`: instância de `RetrievalQA` já configurada com `llm` e `retriever`.

Notas:
- `RetrievalQA.invoke` aceita um dicionário com a chave "query".
- Dependendo da versão do LangChain/chain, o retorno pode ser:
  - um `dict` com a chave "result" (mais comum), ou
  - uma string direta (fallback tratado abaixo).
"""

query_1 = "O que significa churn em análise de clientes?"
query_2 = "Como funciona o pipeline de RAG?"
query_3 = "Explique o que é LangChain."

print("\nPerguntas e respostas no pipeline RAG:\n")

for query in [query_1, query_2, query_3]:
    print(f"Pergunta: {query}")
    # Uso moderno do LangChain (evita DeprecationWarning do .run)
    out = qa_chain.invoke({"query": query})
    resposta = out.get("result") if isinstance(out, dict) else out
    print(f"Resposta: {resposta}\n{'-'*60}")


/tmp/ipython-input-602582006.py:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  resposta = qa_chain.run(query)



Perguntas e respostas no pipeline RAG:

Pergunta: O que significa churn em análise de clientes?
Resposta: NPS, or Net Promoter Score, mede a lealdade dos clientes através da pergunta: 'Você recomendaria nossa empresa a um amigo?'
------------------------------------------------------------
Pergunta: Como funciona o pipeline de RAG?
Resposta: criaço de embeddings, uso do retriever para buscar documents relevantes e geraço da resposta pelo LLM
------------------------------------------------------------
Pergunta: Explique o que é LangChain.
Resposta: a biblioteca para construir aplicaçes que usam modelos de linguagem large (LLMs) integrados a outras fontes de dados
------------------------------------------------------------


# **8. CONCLUSÃO**

In [9]:
"""
Imprime um resumo do que foi feito no notebook.

Pontos-chave:
- Geração de embeddings semânticos com HuggingFaceEmbeddings (ou OpenAIEmbeddings, se configurado).
- Indexação FAISS para busca vetorial eficiente.
- Uso de um retriever (similaridade) para selecionar contexto relevante.
- Combinação com um LLM (OpenAI ou local via Transformers) para respostas fundamentadas (RAG).
"""

print("""
Neste notebook você viu como:
- Criar embeddings semânticos para documentos usando HuggingFaceEmbeddings (ou OpenAIEmbeddings, se preferir).
- Indexar documentos em um vetor store FAISS para busca eficiente.
- Usar o retriever para buscar os documentos mais relevantes para uma query.
- Passar o conteúdo recuperado para um LLM gerar respostas contextuais.
Este é o fluxo típico da pipeline RAG com LangChain: embeddings → retriever → LLM.
""")


Neste notebook você viu como:
- Criar embeddings semânticos para documentos usando OpenAIEmbeddings.
- Indexar documentos em um vetor store FAISS para busca eficiente.
- Usar o retriever para buscar os documentos mais relevantes para uma query.
- Passar o conteúdo recuperado para um LLM gerar respostas contextuais e contextualizadas.
Este é o fluxo típico da pipeline RAG com LangChain: embeddings → retriever → LLM.

